In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jul 27 09:49:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Fine-tuning XLSR-Wav2Vec2 for Multi-Lingual ASR with 🤗 Transformers**

In [3]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.11.3
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer


In [4]:
from huggingface_hub import notebook_login

notebook_login()


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [5]:
pip install git-lfs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Create `Wav2Vec2CTCTokenizer`

In [6]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("common_voice", "pa-IN", split="train+validation")
common_voice_test = load_dataset("common_voice", "pa-IN", split="test")

Downloading:   0%|          | 0.00/4.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/69.7M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/pa-IN/6.1.0/5693bfc0feeade582a78c2fb250bc88f52bd86f0a7f1bb22bfee67e715de30fd. Subsequent calls will reuse this data.


Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/pa-IN/6.1.0/5693bfc0feeade582a78c2fb250bc88f52bd86f0a7f1bb22bfee67e715de30fd)


In [7]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,ਉਹ ਉਸ ਵੱਲ ਪਿੱਠ ਕਰੀ ਪਈ ਸੀ
1,ਮਾਸ ਦੀ ਨਸਲ ਤਾ ਇੱਕ ਹੈ
2,ਆਪਣਾ ਲੈਟਰ ਪੈਡ ਬਣਾਇਆ ਜਾ ਸਕਦਾ ਹੈ
3,ਉਹਨਾਂ ਦਿਨਾਂ ਦੀਆਂ ਯਾਦਾਂ ਅਭੁੱਲ ਹਨ
4,ਪਰ ਇਸ ਵਿਚ ਮੇਰਾ ਕੀ ਕਸੂਰ ਹੈ
5,ਕੋਈ ਨਜ਼ਰ ਵੀ ਐਸੀ ਦੇਖੀ ਨਾ
6,ਤਿਹਾੜ ਦੀਆਂ ਕੁੱਲ ਛੇ ਜੇਲ੍ਹਾਂ ਹਨ
7,ਫ਼ਰਜ਼ੀ ਪੂਜਾ ਦਾ ਢੋਂਗ ਰਚਾ ਨਾ ਬਹਿ
8,ਚਲੋ ਇਨ੍ਹਾਂ ਲੀਡਰਾਂ ਭੁੱਲ ਮੰਨੀ ਤਾਂ ਸਹੀ
9,ਪਰ ਆਦੀ ਹੁਣ ਹਿੰਦੀ ਅਤੇ ਅੰਗਰੇਜ਼ੀ ਵੀ ਸਿੱਖ ਰਹੀ ਹੈ


In [10]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

In [11]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x7f059f7f84d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [12]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))


,sentence
0,ਊਠਾਂ ਬੈਲਾਂ ਦੀ ਸਵਾਰੀ ਨਾ ਬਿਜਲੀ ਦੀ ਚਿੰਤਾ
1,ਮੋਰੀ ਰੁਣ ਝੁਣ ਲਾਇਆ ਭੈਣੇ ਸਾਵਣ ਆਇਆ
2,ਉਨਾਂ ਕਿਹਾ ਕਿ ਕੁੱਝ ਗਾਹਕ ਵੀ ਪੈਸੇ ਲੇਟ ਕਰ ਦਿੰਦੇ ਹਨ
3,ਜੋਤੀ ਬੌਣੇਪਨ ਦੀ ਬਿਮਾਰੀ ਤੋਂ ਪੀੜਤ ਹੈ
4,ਜਿਸ ਸੱਭ ਦਾ ਨਤੀਜਾ ਕਸ਼ਮੀਰੀ ਲੋਕਾਂ ਨੂੰ ਭੁਗਤਣਾ ਪਿਆ ਹੈ
5,ਬਿਲਕੁਲ ਨਹੀ ਨਿਕਲਦੀ ਪਰ ਮੈ ਕਿਆਰਾ ਮੋੜ ਆਵਾਂ
6,ਮੈਂ ਅੰਤਰੀਵ ਵਿਚ ਝਾਤੀ ਮਾਰਨ ਦਾ ਸਬੱਬ
7,ਖੁਸ਼ ਰਹੋ ਜਿਸ ਦੇ ਕਿ ਤੁਸੀਂ ਹੱਕਦਾਰ ਹੋ
8,ਇਹ ਸਾਡੇ ਪਿੰਡ ਦਾ ਹੀ ਸੀ
9,ਨਾਂਗ ਵਾਂਗ ਭੋਰਾਂ ਨਹੀਂ ਲਿਹਾਜ਼ ਕਰਦੀ


In [13]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [14]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [16]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 23,
 'ਂ': 16,
 'ਅ': 57,
 'ਆ': 21,
 'ਇ': 17,
 'ਈ': 25,
 'ਉ': 27,
 'ਊ': 2,
 'ਏ': 41,
 'ਐ': 45,
 'ਓ': 1,
 'ਔ': 5,
 'ਕ': 12,
 'ਖ': 8,
 'ਗ': 48,
 'ਘ': 50,
 'ਚ': 61,
 'ਛ': 37,
 'ਜ': 42,
 'ਝ': 14,
 'ਟ': 55,
 'ਠ': 60,
 'ਡ': 38,
 'ਢ': 30,
 'ਣ': 3,
 'ਤ': 44,
 'ਥ': 36,
 'ਦ': 33,
 'ਧ': 58,
 'ਨ': 18,
 'ਪ': 6,
 'ਫ': 29,
 'ਬ': 7,
 'ਭ': 39,
 'ਮ': 59,
 'ਯ': 47,
 'ਰ': 0,
 'ਲ': 32,
 'ਲ਼': 20,
 'ਵ': 46,
 'ਸ਼': 49,
 'ਸ': 22,
 'ਹ': 34,
 '਼': 51,
 'ਾ': 19,
 'ਿ': 31,
 'ੀ': 11,
 'ੁ': 24,
 'ੂ': 28,
 'ੇ': 52,
 'ੈ': 13,
 'ੋ': 9,
 'ੌ': 26,
 '੍': 53,
 'ਖ਼': 15,
 'ਗ਼': 4,
 'ਜ਼': 43,
 'ੜ': 56,
 'ਫ਼': 40,
 'ੰ': 54,
 'ੱ': 35,
 '…': 10}

In [17]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [18]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

64

In [19]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [20]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [21]:
repo_name = "wav2vec2-large-xls-r-300m-panjabi-colab"

In [22]:
tokenizer.push_to_hub(repo_name)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:43: FutureWarning: Pass token='wav2vec2-large-xls-r-300m-panjabi-colab' as keyword args. From version 0.8 passing these as positional arguments will result in an error
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:644: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/AlphaNinja27/wav2vec2-large-xls-r-300m-panjabi-colab into local empty directory.
To https://huggingface.co/AlphaNinja27/wav2vec2-large-xls-r-300m-panjabi-colab
   b4c1fad..66fbe70  main -> main



'https://huggingface.co/AlphaNinja27/wav2vec2-large-xls-r-300m-panjabi-colab/commit/66fbe70ef1367743a90544ce84962bcf069a6a82'

### Create `Wav2Vec2FeatureExtractor`

In [23]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [24]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
 #processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-panjabi")

# **Preprocess** **Data**

In [25]:
common_voice_train[0]["path"]

'common_voice_pa-IN_21916812.mp3'

In [26]:
common_voice_train[0]["audio"]


{'array': array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.1980534e-05,
        1.7583370e-05, 2.4676323e-05], dtype=float32),
 'path': 'cv-corpus-6.1-2020-12-11/pa-IN/clips/common_voice_pa-IN_21916812.mp3',
 'sampling_rate': 48000}

In [27]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))


In [28]:
common_voice_train[0]["audio"]


{'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -5.7883699e-06, -1.3625722e-05,  1.2423844e-05], dtype=float32),
 'path': 'cv-corpus-6.1-2020-12-11/pa-IN/clips/common_voice_pa-IN_21916812.mp3',
 'sampling_rate': 16000}

In [30]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)


ਬਿਮਾਰ ਤੰਦਰੁਸਤ ਅਮੀਰ ਗਰੀਬ ਹੋ ਗਿਆ 


In [31]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: ਇਸ ਨੈਤਿਕ ਜਿ਼ੰਮੇਵਾਰੀ ਸ਼ਬਦ ਦਾ ਮਤਲਬ ਕੀ ਹੈ 
Input array shape: (76800,)
Sampling rate: 16000


In [32]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [33]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

# **Training**

# Set-up **Trainer**

In [34]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [35]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [36]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [37]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [38]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [39]:
model.freeze_feature_extractor()

In [40]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [41]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

/content/wav2vec2-large-xls-r-300m-panjabi-colab is already a clone of https://huggingface.co/AlphaNinja27/wav2vec2-large-xls-r-300m-panjabi-colab. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp fp16 backend


### Training

In [55]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length.
***** Running training *****
  Num examples = 255
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 240
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=240, training_loss=0.23091996510823568, metrics={'train_runtime': 1094.2908, 'train_samples_per_second': 6.991, 'train_steps_per_second': 0.219, 'total_flos': 1.2258586474308403e+18, 'train_loss': 0.23091996510823568, 'epoch': 30.0})

In [56]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-large-xls-r-300m-panjabi-colab
Configuration saved in wav2vec2-large-xls-r-300m-panjabi-colab/config.json
Model weights saved in wav2vec2-large-xls-r-300m-panjabi-colab/pytorch_model.bin
Configuration saved in wav2vec2-large-xls-r-300m-panjabi-colab/preprocessor_config.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/1.18G [00:00<?, ?B/s]

Upload file runs/Jul27_10-17-18_388b47d469c9/1658919027.1415005/events.out.tfevents.1658919027.388b47d469c9.72…

Upload file runs/Jul27_10-17-18_388b47d469c9/events.out.tfevents.1658919027.388b47d469c9.72.2:  75%|#######4  …

Upload file runs/Jul27_10-17-18_388b47d469c9/1658920611.0556965/events.out.tfevents.1658920611.388b47d469c9.72…

Upload file runs/Jul27_10-17-18_388b47d469c9/events.out.tfevents.1658920611.388b47d469c9.72.4:  75%|#######4  …

To https://huggingface.co/AlphaNinja27/wav2vec2-large-xls-r-300m-panjabi-colab
   66fbe70..deaa5bb  main -> main

Dropping the following result as it does not have all the necessary field:
{'dataset': {'name': 'common_voice', 'type': 'common_voice', 'args': 'pa-IN'}}
To https://huggingface.co/AlphaNinja27/wav2vec2-large-xls-r-300m-panjabi-colab
   deaa5bb..5e68968  main -> main



'https://huggingface.co/AlphaNinja27/wav2vec2-large-xls-r-300m-panjabi-colab/commit/deaa5bb5f32f6533e2a6ee50784dca874616e4bd'

# **Evaluation**

In [57]:
model = Wav2Vec2ForCTC.from_pretrained(repo_name).to("cuda")
processor = Wav2Vec2Processor.from_pretrained(repo_name)

loading configuration file wav2vec2-large-xls-r-300m-panjabi-colab/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-xls-r-300m",
  "activation_dropout": 0.0,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.0,
  "grad

In [58]:
input_dict = processor(common_voice_test[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


In [59]:
common_voice_test_transcription = load_dataset("common_voice", "pa-IN", data_dir="./cv-corpus-6.1-2020-12-11", split="test")

Using custom data configuration pa-IN-24242e0bc11eddb3
Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/pa-IN-24242e0bc11eddb3/6.1.0/5693bfc0feeade582a78c2fb250bc88f52bd86f0a7f1bb22bfee67e715de30fd)


In [60]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[0]["sentence"].lower())

Prediction:
ਉਨਾਂ ਨਣੇ ਸਾਰੇ ਤਿਰਵੇ ਵਖਰੀ ਕਸਮਦ ਕੀਤੇਹ ਹਨ

Reference:
ਉਨ੍ਹਾਂ ਨੇ ਸਾਰੇ ਤਜ਼ਰਬੇ ਵੱਖਰੀ ਕਿਸਮ ਦੇ ਕੀਤੇ ਹਨ
